# In the main branch, this file should be no executed and with no parameters set.
# In the development branch, it must be fully executed and must include tests.

GOAL: calculate the physical quantities (magnetization, susceptibility, binder cumulant, and possible errors) from the raw results stored at an SQLite DB.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

## Parameters

In [ ]:
# Set the filename of the raw results database.
# By raw I mean coming directly from simulations.
#
# Example: 
#
# db_filename = 'bubble_filtering__8_neighbors_squared_network.db'

db_filename = 

<br>
Hereafter, the code should not be modified at the standard usage.

## Definitions

Here I write the definitions used to obtain the relevant physical quantities: magnetization, susceptibility, and Binder cumulant.

<b>Conventions</b>:

    - Mean of a random variable X:
<center><font size="5">$<X>_{t} = \hat{\mu}_{t}(X) = \frac{1}{T} \sum_{t=1}^{T} X_{t}$</font></center>
    
    - Standard deviation:
<center><font size="5">$std_{t}(X) = \hat{\sigma}_{t}(X) = \sqrt{\frac{1}{T-1}\sum_{t=1}^{T} (X_{t} - \hat{\mu}(X))^2}$</font></center>

    - Magnetization for a given time t and configuration c, as a function of the visibility v, network size N, and noise q:
<center><font size="5">$M_{tc}(v, N, q) = m_{tc}$</font></center>

<b>Physical Quantities</b>:

    - Magnetization:
<center><font size="5">$M(v, N, q) = \hat{\mu}_{c}(\hat{\mu}_{t}(m_{tc}))$</font></center>

    - Magnetization error:
<center><font size="5">$\Delta M(v, N, q) = \hat{\sigma}_{c}(\hat{\mu}_{t}(m_{tc}))$</font></center>

    - Susceptibility:
<center><font size="5">$\chi(v, N, q) = \hat{\mu}_{c}\left(N\left\{\hat{\mu}_{t}(m_{tc}^{2}) - [\hat{\mu}_{t}(m_{tc})]^{2} \right\}\right)$</font></center>

    - Susceptibility error:
<center><font size="5">$\Delta \chi(v, N, q) = \hat{\sigma}_{c}\left(N\left\{\hat{\mu}_{t}(m_{tc}^{2}) - [\hat{\mu}_{t}(m_{tc})]^{2} \right\}\right)$</font></center>

    - Binder cumulant:
<center><font size="5">$U(v, N, q) = \hat{\mu}_{c}\left( 1 - \hat{\mu}_{t}(m_{tc}^{4})\hspace{1mm}/\hspace{1mm} 3\hat{\mu}_{t}(m_{tc}^{2}) \right)$</font></center>

    - Binder cumulant error:
<center><font size="5">$\Delta U(v, N, q) = \hat{\sigma}_{c}\left( 1 - \hat{\mu}_{t}(m_{tc}^{4})\hspace{1mm}/\hspace{1mm} 3\hat{\mu}_{t}(m_{tc}^{2}) \right)$</font></center>

Remember that np.std computes the biased standard deviation while pandas calculates the non-biased.

## Functions

In [ ]:
def calc_X_and_U(df):
    '''
    Calculate Susceptibility and Binder cumulant for each configuration in the results.
    '''
    
    df = df.sort_values(['v', 'N', 'q', 'net', 'rep'])
    df['X'] = df['N']*(df['m2T'] - df['mT']**2)
    df['X_noMod'] = df['N']*(df['m2T'] - df['mT_noMod']**2)
    df['U'] = 1 - df['m4T']/(3*df['m2T']**2)
    df['u'] = -np.log(1 - 3*df['U']/2)
    
    return df

In [ ]:
def calc_averages(columnLabel, results_df, new_label = ''):
    '''
    Calculate the means for grouped values of columnLabel.
    
    Parameters:
    columnLabel should be in ['mT', 'm2T', 'm4T'].
    results_df contains the temporal averages.
    
    Returns a pandas series of replications and networks averages.
    '''
    
    # Do the mean over the replications
    x = pd.DataFrame(results_df.groupby(['v','N','q'])[columnLabel].mean())
    
    if new_label != '':
        x.rename({columnLabel: new_label},axis=1,inplace=True)
    
    return x



def calc_avg_std(df_grouped, columnLabel, func, new_label = ''):
    '''
    Calculate the average or standard deviation for grouped values of columnLabel.
    
    Parameters:
    columnLabel should be in ['mT', 'mT_noMod', 'X', 'X_noMod', 'U', 'u'].
    func should be in ['mean', 'std'].
    df_grouped contains the temporal averages grouped by v,N,q.
    
    Returns a pandas series of configurations averages or standard deviations.
    '''
    
    # Do the mean over the replications
    if func == 'mean':
        x = pd.DataFrame(df_grouped[columnLabel].mean())
    elif func == 'std':
        x = pd.DataFrame(df_grouped[columnLabel].std())
    else:
        raise Exception("Parameter func should be in ['mean', 'std']")
    
    if new_label != '':
        x.rename({columnLabel: new_label},axis=1,inplace=True)
    
    return x

In [ ]:
# Calculate the physical interesting quantities

def calculate_physical_quantities(df):
    '''
    
    '''
    
    df_grouped = df.groupby(['v','N','q'])

    M       = calc_avg_std(df_grouped, 'mT'      , 'mean', 'M')
    M_noMod = calc_avg_std(df_grouped, 'mT_noMod', 'mean', 'M_noMod')
    X       = calc_avg_std(df_grouped, 'X'       , 'mean')
    X_noMod = calc_avg_std(df_grouped, 'X_noMod' , 'mean')
    U       = calc_avg_std(df_grouped, 'U'       , 'mean')
    u       = calc_avg_std(df_grouped, 'u'       , 'mean')

    M_err       = calc_avg_std(df_grouped, 'mT'      , 'std', 'M_err')
    M_noMod_err = calc_avg_std(df_grouped, 'mT_noMod', 'std', 'M_noMod_err')
    X_err       = calc_avg_std(df_grouped, 'X'       , 'std', 'X_err')
    X_noMod_err = calc_avg_std(df_grouped, 'X_noMod' , 'std', 'X_noMod_err')
    U_err       = calc_avg_std(df_grouped, 'U'       , 'std', 'U_err')
    u_err       = calc_avg_std(df_grouped, 'u'       , 'std', 'u_err')

    pq = M.join([M_err, M_noMod, M_noMod_err, 
                X, X_err, X_noMod, X_noMod_err, 
                U, U_err, u, u_err])
    
    return pq



def calculate_physical_quantities__OLD(df):
    
    
    phys_quant = pd.DataFrame(calc_averages('mT', df))
    phys_quant.reset_index(inplace=True)
    phys_quant.rename({'mT': 'M'},axis=1,inplace=True)

    m2df = calc_averages('m2T', df)
    m4df = calc_averages('m4T', df)
    
    phys_quant['M_err'] = np.array(m2df.m2T) - phys_quant['M']**2
    phys_quant['M_noMod'] = np.array(calc_averages('mT_noMod', df))
    phys_quant['M_noMod_err'] = np.array(m2df.m2T) - phys_quant['M_noMod']**2

    phys_quant['X'] = phys_quant['N']*phys_quant['M_err']
    phys_quant['X_noMod'] = phys_quant['N']*phys_quant['M_noMod_err']

    phys_quant['U'] = 1 - np.array(m4df.m4T)/(3*np.array(m2df.m2T)**2)

    phys_quant['u'] = -np.log(1-3*phys_quant['U']/2)
    
    return phys_quant

## Main

In [ ]:
# Read the database with pandas

conn = sqlite3.connect('../results_databases/' + db_filename)
df = pd.read_sql('SELECT * FROM temporal_mean_results', conn)
conn.close()

# Calculate the susceptibility and the binder cumulant.
df = calc_X_and_U(df)

# Calculate the physical quantities.
phys_quant = calculate_physical_quantities(df)
phys_quant.reset_index(inplace=True)

In [ ]:
# Persist the data frame

# Comment the exception raising to persist the data frame
raise Exception('Do not execute this cell accidentally.')

phys_quant.to_csv(
    '../results_databases/' + db_filename[:-3] + '__MXU.csv')

<br><br>
## Testing

In [ ]:
#The test below is just a check. 
# Once I know some plot for the physical quantities 
# I am just doing a visual inspection.

In [ ]:
phys_quant.head()

In [ ]:
x = phys_quant[phys_quant.v == 1]
N = x.N.unique()
x.set_index(['N','q'], inplace=True)
for n in N:
    x.loc[n].M.plot(label=n)
plt.title('Magnetization (absolute value before temporal avg.')
plt.legend(loc=(1.05,0))
plt.show()

for n in N:
    x.loc[n].u.plot(label=n)
plt.title('Binder cumulant')
plt.legend(loc=(1.05,0))
plt.show()

In [ ]:
x = phys_quant[phys_quant.v == 0.4]
N = x.N.unique()
x.set_index(['N','q'], inplace=True)
for n in N:
    x.loc[n].M.plot(label=n)
plt.title('Magnetization (absolute value before temporal avg.')
plt.legend(loc=(1.05,0))
plt.show()

for n in N:
    x.loc[n].u.plot(label=n)
plt.title('Binder cumulant')
plt.legend(loc=(1.05,0))
plt.show()

In [ ]:
x = phys_quant[phys_quant.v == 0.25]
N = x.N.unique()
x.set_index(['N','q'], inplace=True)
for n in N:
    x.loc[n].M.plot(label=n)
plt.title('Magnetization (absolute value before temporal avg.')
plt.legend(loc=(1.05,0))
plt.show()

for n in N:
    x.loc[n].u.plot(label=n)
plt.title('Binder cumulant')
plt.legend(loc=(1.05,0))
plt.show()